In [1]:
# Install and import the necessary libraries

!pip install torch
!pip install -q -U accelerate peft bitsandbytes transformers trl einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [2]:
!pip install datasets

In [3]:
import os
import torch
from datasets import load_dataset
from datasets import load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer

In [4]:
# Model
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

# Dataset
dataset = load_dataset('stanfordnlp/snli')

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
def combine_texts(examples):
    examples['text'] = examples['premise'] + " " + examples['hypothesis']  # Combine premise and hypothesis
    return examples

In [6]:
train_data = dataset['train'].select(range(0, 550000, 550))  # 1000 samples
val_data = dataset['validation'].select(range(0, 10000, 100))  # 100 samples
test_data = dataset['test'].select(range(0, 10000, 100))  # 100 samples

In [7]:
val_data[99]

{'premise': 'Two boys, one wearing green, the other in yellow, are playing with sidewalk chalk, drawing pictures on the ground.',
 'hypothesis': 'There are two boys making chalk drawings outside.',
 'label': 0}

In [8]:
train_data = dataset['train'].map(combine_texts)
val_data = dataset['validation'].map(combine_texts)
test_data = dataset['test'].map(combine_texts)

Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
train_data[9]

{'premise': 'An older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.',
 'hypothesis': 'An older man drinks his juice as he waits for his daughter to get off work.',
 'label': 1,
 'text': 'An older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background. An older man drinks his juice as he waits for his daughter to get off work.'}

In [10]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [11]:
# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    revision="refs/pr/23" #the main version of Phi-2 doesn’t support gradient checkpointing (while training this model)
)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [12]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [13]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    return f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"

In [11]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 5,
    fp16 = False,
    bf16 = False,
    dataloader_pin_memory = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 4,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 1e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine",
    do_train = True,
    do_eval = True,
    eval_strategy = 'epoch',
    logging_strategy = 'epoch',
    save_strategy='epoch',
    max_steps = 500,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

# LoRA configuration
peft_config = LoraConfig(
    r=32,                   #default=8
    lora_alpha= 16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["out_proj"]#["Wqkv", "out_proj"] #['q_proj', 'dense'] #["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ]
)
print(print_trainable_parameters(model))

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=peft_config,
    dataset_text_field="premise",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 0 || all params: 1521392640 || trainable%: 0.0


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# inputs = inputs.to(device)

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=2560, bias=False)
            )
            (lora_embedding_A): Paramete

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,12.527100,3.010164


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


TrainOutput(global_step=500, training_loss=12.5270546875, metrics={'train_runtime': 1852.7625, 'train_samples_per_second': 4.318, 'train_steps_per_second': 0.27, 'total_flos': 1886635935866880.0, 'train_loss': 12.5270546875, 'epoch': 0.014541435821373003})

In [14]:
trainer.model.save_pretrained(new_model)

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


In [15]:
trainer.save_model('./final_model')

You are using a model of type phi to instantiate a model of type phi-msft. This is not supported for all configurations of models and can yield errors.


In [16]:
wandb.finish()

NameError: name 'wandb' is not defined

In [16]:
# #Check training results with tensorboard
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [15]:
# Run text generation pipeline with our model
logging.set_verbosity(logging.CRITICAL)

prompt = "What does an old man do and why?"
instruction = f"### Instruction: {prompt} "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])



An old man is a person who is very old. He may have lived for a very long time and has many stories to tell. He may also have a lot of wisdom and knowledge to share with others.

- Exercise: Can you think of any reasons why an old man might be important? 

Answer: An old man may be important because he has lived a long life and has many experiences to share. He may also have a lot of wisdom and knowledge to pass on to others.

- Exercise: Can you think of any ways an old man can help others? 

Answer: An old man can help others by sharing his experiences and wisdom, and by being a role model for younger generations.

- Exercise: Can you think of any ways an old man can make a difference in the world? 

Answer: An old man can make a difference in the world


In [17]:
# Reload model and merge it with LoRA parameters
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    cache_dir="",
    device_map={"": 0},
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Target modules {'out_proj'} not found in the base model. Please check the target modules and try again.

In [18]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [19]:
model.save_pretrained("phi_2_new_model")
tokenizer.save_pretrained("phi_2_new_tokenizer")

# To load the saved model and tokenizer
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("phi_2_new_model")
tokenizer = AutoTokenizer.from_pretrained("phi_2_new_tokenizer")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [7]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from transformers import pipeline

import evaluate

# Load the test dataset (assuming it's already loaded and preprocessed as 'test_data')

# Load both pretrained and fine-tuned models
base_model = "microsoft/phi-2"
new_model = "phi-2-medquad"

pretrained_model = AutoModelForCausalLM.from_pretrained(base_model, device_map={"": 0})
fine_tuned_model = AutoModelForCausalLM.from_pretrained(new_model, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Initialize the accuracy metric from the `evaluate` library
metric = evaluate.load("accuracy")

# Set up pipelines for inference
pretrained_pipeline = pipeline("text-classification", model=pretrained_model, tokenizer=tokenizer, device=0)
fine_tuned_pipeline = pipeline("text-classification", model=fine_tuned_model, tokenizer=tokenizer, device=0)

# Evaluate Pretrained Model
pretrained_predictions = []
for example in test_data:
    result = pretrained_pipeline(example["text"])
    predicted_label = result[0]["label"]
    pretrained_predictions.append(int(predicted_label.split("_")[1]))

# Evaluate Fine-Tuned Model
fine_tuned_predictions = []
for example in test_data:
    result = fine_tuned_pipeline(example["text"])
    predicted_label = result[0]["label"]
    fine_tuned_predictions.append(int(predicted_label.split("_")[1]))

# Compute accuracy for each model
pretrained_accuracy = metric.compute(predictions=pretrained_predictions, references=test_data["label"])["accuracy"]
fine_tuned_accuracy = metric.compute(predictions=fine_tuned_predictions, references=test_data["label"])["accuracy"]

print(f"Pretrained Model Accuracy: {pretrained_accuracy}")
print(f"Fine-Tuned Model Accuracy: {fine_tuned_accuracy}")

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 17.06 MiB is free. Process 293266 has 14.73 GiB memory in use. Of the allocated memory 14.63 GiB is allocated by PyTorch, and 997.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)